In [1]:
cd ..

/Users/kimyoungh/GoogleDrive/rich/main/rich


In [2]:
import numpy as np
import pandas as pd

from data_processing.price_factor_processor import PriceFactorProcessor
from data_processing.processing_trader_training_data import ProcessingTraderTrainingData

%load_ext autoreload
%autoreload 2

In [3]:
returns = pd.read_csv('index_strategy_returns.csv', header=0, index_col=0)
returns.index = returns.index.astype(str)

In [4]:
returns['K200_2X'] = returns['K200'] * 2.
returns['KQ_2X'] = returns['KQ'] * 2.
returns['K200_i_2X'] = returns['K200_i'] * 2.

In [5]:
global_index_series = pd.read_parquet('trading_data/global_index_series.pq')

gindex = global_index_series.index.to_pydatetime()
gindex = pd.Series(gindex).apply(lambda x: x.strftime('%Y%m%d'))

In [6]:
gindex = gindex.values

global_index = pd.DataFrame(global_index_series.values,
                            index=gindex,
                            columns=global_index_series.columns.values)

spos = np.argwhere(returns.index == global_index.index[1]).item()
gepos = np.argwhere(global_index.index == returns.index[-1]).item()

total_index = global_index.index.append(returns.index).unique().sort_values()

global_index_t = pd.DataFrame(global_index,
    index=total_index).ffill().loc[returns.index]
greturns = global_index_t.pct_change().shift(1).dropna()
returns = returns.loc[greturns.index]

returns_train = returns.iloc[:-550]
returns_test = returns.iloc[-249-550:]

greturns_train = greturns.iloc[:-550]
greturns_test = greturns.iloc[-249-550:]

In [7]:
print(returns_train.shape)
print(returns_test.shape)
print(greturns_train.shape)
print(greturns_test.shape)

(3468, 7)
(799, 7)
(3468, 173)
(799, 173)


In [9]:
pttd_train = ProcessingTraderTrainingData(returns_train,
    greturns_train, window=60)
pttd_test = ProcessingTraderTrainingData(returns_test,
    greturns_test, window=60)

In [10]:
factors_train, gfactors_train, weights_train, stock_codes_train,\
    trade_dates_train, gtrade_dates_train, factors_index_train,\
    gfactors_index_train,\
        return_series_train = pttd_train.generate_target_data()

/Users/kimyoungh/miniforge3/envs/pytorch_m1/lib/python3.8/site-packages/cvxpy/problems/problem.py:1296: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [11]:
factors_test, gfactors_test, weights_test, stock_codes_test,\
    trade_dates_test, gtrade_dates_test, factors_index_test,\
    gfactors_index_test,\
        return_series_test = pttd_test.generate_target_data()

In [12]:
print(factors_train.shape)
print(gfactors_train.shape)
print(weights_train.shape)
print(stock_codes_train.shape)
print(trade_dates_train.shape)
print(gtrade_dates_train.shape)
print(factors_index_train.shape)
print(gfactors_index_train.shape)
print(return_series_train.shape)

(3159, 7, 69)
(3159, 173, 69)
(3159, 7)
(7,)
(3159,)
(3219,)
(3159, 7, 2)
(3159, 173, 2)
(3159, 60, 7)


In [13]:
print(factors_test.shape)
print(gfactors_test.shape)
print(weights_test.shape)
print(stock_codes_test.shape)
print(trade_dates_test.shape)
print(gtrade_dates_test.shape)
print(factors_index_test.shape)
print(gfactors_index_test.shape)
print(return_series_test.shape)

(490, 7, 69)
(490, 173, 69)
(490, 7)
(7,)
(490,)
(550,)
(490, 7, 2)
(490, 173, 2)
(490, 60, 7)


In [14]:
data_path = "./train_data/trader/"

np.save(data_path+"factors_train.npy", factors_train, allow_pickle=True)
np.save(data_path+"gfactors_train.npy", gfactors_train, allow_pickle=True)
np.save(data_path+"weights_train.npy", weights_train, allow_pickle=True)
np.save(data_path+"stock_codes.npy", stock_codes_train, allow_pickle=True)
np.save(data_path+"trade_dates_train.npy", trade_dates_train,
    allow_pickle=True)
np.save(data_path+"factors_index_train.npy", factors_index_train,
    allow_pickle=True)
np.save(data_path+"gfactors_index_train.npy", gfactors_index_train,
    allow_pickle=True)
np.save(data_path+"return_series_train.npy", return_series_train,
    allow_pickle=True)

In [15]:
np.save(data_path+"factors_test.npy", factors_test, allow_pickle=True)
np.save(data_path+"gfactors_test.npy", gfactors_test, allow_pickle=True)
np.save(data_path+"weights_test.npy", weights_test, allow_pickle=True)
np.save(data_path+"stock_codes_test.npy", stock_codes_test, allow_pickle=True)
np.save(data_path+"trade_dates_test.npy", trade_dates_test,
    allow_pickle=True)
np.save(data_path+"factors_index_test.npy", factors_index_test,
    allow_pickle=True)
np.save(data_path+"gfactors_index_test.npy", gfactors_index_test,
    allow_pickle=True)
np.save(data_path+"return_series_test.npy", return_series_test,
    allow_pickle=True)

In [16]:
print(factors_train.shape)
print(weights_train.shape)
print(gfactors_train.shape)
print(return_series_train.shape)

(3159, 7, 69)
(3159, 7)
(3159, 173, 69)
(3159, 60, 7)


In [17]:
print(factors_test.shape)
print(weights_test.shape)
print(gfactors_test.shape)
print(return_series_test.shape)

(490, 7, 69)
(490, 7)
(490, 173, 69)
(490, 60, 7)
